In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
data = pd.read_csv('spam.csv',encoding='latin-1')

data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":'label', "v2":'text'})
print(data.head())
tags = data["label"]
texts = data["text"]

  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


## To do: develop an accurate simple neural network model for spam classification (no LSTM, CNN, etc.)

In [2]:
data.shape

(5572, 2)

In [3]:
data.columns

Index(['label', 'text'], dtype='object')

In [4]:
data2=pd.get_dummies(data['label'],drop_first=False)
data2

,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
5567,0,1
5568,1,0
5569,1,0
5570,1,0


In [5]:

sentences = data['text'].tolist()
labels = data2['spam'].tolist()

training_size = int(len(sentences) * 0.8)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = 600, oov_token = "<OOV>")
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = 60, padding = 'post', truncating = 'post')

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = 60, padding = 'post', truncating = 'post')

In [7]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(600, 16, input_length = 60),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 16)            9600      
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 6)                 102       
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 9,709
Trainable params: 9,709
Non-trainable params: 0
_________________________________________________________________


In [8]:
num_epochs = 100
history=model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/100
140/140 [==============================] - 3s 8ms/step - loss: 0.5624 - accuracy: 0.8647 - val_loss: 0.3965 - val_accuracy: 0.8700
Epoch 2/100
140/140 [==============================] - 1s 4ms/step - loss: 0.3487 - accuracy: 0.8649 - val_loss: 0.3048 - val_accuracy: 0.8700
Epoch 3/100
140/140 [==============================] - 1s 4ms/step - loss: 0.2802 - accuracy: 0.8652 - val_loss: 0.2290 - val_accuracy: 0.8717
Epoch 4/100
140/140 [==============================] - 1s 5ms/step - loss: 0.2027 - accuracy: 0.9028 - val_loss: 0.1611 - val_accuracy: 0.9534
Epoch 5/100
140/140 [==============================] - 1s 4ms/step - loss: 0.1488 - accuracy: 0.9545 - val_loss: 0.1209 - val_accuracy: 0.9668
Epoch 6/100
140/140 [==============================] - 1s 4ms/step - loss: 0.1108 - accuracy: 0.9684 - val_loss: 0.0919 - val_accuracy: 0.9749
Epoch 7/100
140/140 [==============================] - 1s 4ms/step - loss: 0.0859 - accuracy: 0.9744 - val_loss: 0.0722 - val_accuracy: 0.9776

Epoch 58/100
140/140 [==============================] - 1s 4ms/step - loss: 0.0081 - accuracy: 0.9984 - val_loss: 0.0757 - val_accuracy: 0.9839
Epoch 59/100
140/140 [==============================] - 1s 8ms/step - loss: 0.0079 - accuracy: 0.9978 - val_loss: 0.0773 - val_accuracy: 0.9830
Epoch 60/100
140/140 [==============================] - 1s 7ms/step - loss: 0.0076 - accuracy: 0.9987 - val_loss: 0.0780 - val_accuracy: 0.9830
Epoch 61/100
140/140 [==============================] - 1s 5ms/step - loss: 0.0070 - accuracy: 0.9984 - val_loss: 0.0805 - val_accuracy: 0.9821
Epoch 62/100
140/140 [==============================] - 1s 6ms/step - loss: 0.0073 - accuracy: 0.9987 - val_loss: 0.0811 - val_accuracy: 0.9830
Epoch 63/100
140/140 [==============================] - 1s 4ms/step - loss: 0.0073 - accuracy: 0.9984 - val_loss: 0.0815 - val_accuracy: 0.9830
Epoch 64/100
140/140 [==============================] - 1s 6ms/step - loss: 0.0071 - accuracy: 0.9984 - val_loss: 0.0829 - val_accuracy:

In [9]:
result = model.evaluate(testing_padded, testing_labels_final)

loss = result[0]
accuracy = result[1]

print(f"[+] Accuracy: {accuracy*100:.2f}%")

35/35 [==============================] - 0s 3ms/step - loss: 0.1267 - accuracy: 0.9803
[+] Accuracy: 98.03%


In [10]:
data3 = pd.read_csv('output_spam.csv',encoding='latin-1')
text_messages = data3['text']

print(text_messages) 

padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(text_messages)
fakes_padded = pad_sequences(sample_sequences, padding = 'post', maxlen = 60)           

classes = model.predict(fakes_padded)

for x in range(len(text_messages)):
  print(text_messages[x])
  print(classes[x])
  print('\n')

0     Feel Yourself That You Are Always Happy.. Slow...
1     staff.science.nus.edu.sg/~phyhcmk/teaching/pc1323
2                           Send me yetty's number pls.
3     Hey so this sat are we going for the intro pil...
4     I got it before the new year cos yetunde said ...
5     Hey we can go jazz power yoga hip hop kb and y...
6     Hey mate. Spoke to the mag people. Weâ°ÃÃ·re...
7                                Morning only i can ok.
8                                  Wat time Ã_ finish?
9                 Shant disturb u anymore... Jia you...
10    4mths half price Orange line rental & latest c...
11    Your opinion about me? 1. Over 2. Jada 3. Kusr...
12    MOON has come to color your dreams, STARS to m...
13    You are a winner U have been specially selecte...
14    Unless it's a situation where YOU GO GURL woul...
15    Awww dat is sweet! We can think of something t...
16                              Bring tat cd don forget
17    Single line with a big meaning::::: \Miss 

In [11]:
spamham = ['spam' if x > 0.1 else 'ham' for x in classes]
outputcsv = pd.DataFrame(spamham, columns = ['label']).to_csv('output_spam.csv')

In [12]:
data4 = pd.read_csv('output_spam.csv')
data4["text"] = text_messages
data4.to_csv('output_spam.csv')

In [13]:
data5 = pd.read_csv('output_spam.csv')
cols=data5.columns
cols=['label','text']
data5=data5[cols]
data5

,label,text
0,ham,Feel Yourself That You Are Always Happy.. Slow...
1,ham,staff.science.nus.edu.sg/~phyhcmk/teaching/pc1323
2,ham,Send me yetty's number pls.
3,ham,Hey so this sat are we going for the intro pil...
4,ham,I got it before the new year cos yetunde said ...
5,ham,Hey we can go jazz power yoga hip hop kb and y...
6,ham,Hey mate. Spoke to the mag people. Weâ°ÃÃ·re...
7,ham,Morning only i can ok.
8,ham,Wat time Ã_ finish?
9,ham,Shant disturb u anymore... Jia you...


In [14]:
data5.to_csv('output_spam.csv')